In [1]:
import numpy as np
import pandas as pd
import datetime
import pickle #pickle序列化对象并保存到磁盘中，并在需要的时候读取出来
import os

In [3]:

############读取所有的文件#########################
def read_files():
    #用户表
    df_user = pd.read_csv("JData_User.csv", encoding='gbk')
    #商品表
    df_product = pd.read_csv("JData_Product.csv", encoding='gbk')
    #评论表
    df_comment = pd.read_csv("JData_Comment.csv", encoding='gbk')
    #行为表
    df_action_1 = pd.read_csv("JData_Action_201602.csv", encoding='gbk')
    df_action_2 = pd.read_csv("JData_Action_201603.csv", encoding='gbk')
    df_action_3 = pd.read_csv("JData_Action_201604.csv", encoding='gbk')
    df_action = pd.concat([df_action_1, df_action_2, df_action_3], axis=0)
        
    return df_user, df_product, df_comment, df_action

df_user, df_product, df_comment, df_action = read_files()

In [4]:
def transform_data(user_df, product_df, comment_df, action_df):
    
    for df in [user_df, product_df, comment_df, action_df]:
        print(df.dtypes)
        print(df.head())
        print("----------------")

    
    print("Processing user_df...")
    user_df[["age", "sex", "user_lv_cd"]] = user_df[["age", "sex", "user_lv_cd"]].fillna(-1)
    user_df["sex"] = user_df["sex"].apply(lambda value: -1 if int(value) == 2 else value)
    user_df["user_reg_tm"] = pd.to_datetime(user_df["user_reg_tm"])
    user_df[["user_id", "age", "sex", "user_lv_cd"]] = user_df[["user_id", "age", "sex", "user_lv_cd"]].apply(pd.to_numeric, errors='coerce').fillna(-1).astype(np.int32)

    
    print("Processing product_df...")
    product_df[["a1", "a2", "a3"]] = product_df[["a1", "a2", "a3"]].fillna(-1)
    product_df[["sku_id", "a1", "a2", "a3", "cate", "brand"]] = product_df[["sku_id", "a1", "a2", "a3", "cate", "brand"]].astype(np.int32)

    
    print("Processing comment_df...")
    comment_df[["comment_num", "has_bad_comment", "bad_comment_rate"]] = comment_df[["comment_num", "has_bad_comment", "bad_comment_rate"]].fillna(0)
    comment_df["dt"] = pd.to_datetime(comment_df["dt"])
    comment_df[["sku_id", "comment_num", "has_bad_comment"]] = comment_df[["sku_id", "comment_num", "has_bad_comment"]].astype(np.int32)
    comment_df["bad_comment_rate"] = comment_df["bad_comment_rate"].astype(np.float32)

    
    print("Processing action_df...")
    action_df["time"] = action_df["time"].apply(lambda t: datetime.datetime.strptime(str(t), "%Y-%m-%d %H:%M:%S"))
    action_df[["user_id", "sku_id", "type", "cate", "brand"]] = action_df[["user_id", "sku_id", "type", "cate", "brand"]].astype(np.int32)

    
    for df in [user_df, product_df, comment_df, action_df]:
        print(df.dtypes)
        print(df.head())
        print("----------------")

    return user_df, product_df, comment_df, action_df

# # 调用函数
# df_user, df_product, df_comment, df_action = transform_data(df_user, df_product, df_comment, df_action)

In [5]:


def save_data(df_user, df_product, df_comment, df_action):
    if not os.path.exists("data.pkl"):
        with open("data.pkl", "wb") as f:
            pickle.dump((df_user, df_product, df_comment, df_action), f)
            
def load_data():
    with open("data.pkl", "rb") as f:
        df_user, df_product, df_comment, df_action = pickle.load(f)
    return df_user, df_product, df_comment, df_action

# # 首先保存数据
# save_data(df_user, df_product, df_comment, df_action)

# 然后加载数据
df_user, df_product, df_comment, df_action = load_data()

In [6]:
def fetch_users_registered_before(user_df, date_str):
    date_threshold = pd.to_datetime(date_str)
    filtered_users = user_df[user_df["user_reg_tm"] < date_threshold]
    return filtered_users

users_registered_before_416 = fetch_users_registered_before(df_user, "2016-04-16")

In [7]:
def exclude_users_by_registration(user_df, filtered_users):
    valid_user_ids = filtered_users["user_id"]
    updated_user_df = user_df[user_df["user_id"].isin(valid_user_ids)]
    return updated_user_df

df_user = exclude_users_by_registration(df_user, users_registered_before_416)

In [8]:
#获取商品评论的特征，以最近的日期为准
def extract_latest_comments_before(comment_df, date_str):
    date_threshold = pd.to_datetime(date_str)
    filtered_comments = comment_df[comment_df["dt"] < date_threshold]
    latest_comment_dates = filtered_comments.groupby("sku_id")["dt"].max().reset_index()
    latest_comments = pd.merge(filtered_comments, latest_comment_dates, on=["sku_id", "dt"], how="inner")
    latest_comments.drop(columns=["dt"], inplace=True)
    return latest_comments

latest_comments_before_416 = extract_latest_comments_before(df_comment, "2016-04-16")

def merge_product_with_comments(product_df, latest_comments):
    merged_df = pd.merge(product_df, latest_comments, on="sku_id", how="left").fillna(0)
    merged_df[["comment_num", "has_bad_comment"]] = merged_df[["comment_num", "has_bad_comment"]].astype(np.int32)
    return merged_df

df_sku = merge_product_with_comments(df_product, latest_comments_before_416)
print(df_sku.head())

   sku_id  a1  a2  a3  cate  brand  comment_num  has_bad_comment  \
0      10   3   1   1     8    489            0                0   
1  100002   3   2   2     8    489            0                0   
2  100003   1  -1  -1     8     30            0                0   
3  100006   1   2   1     8    545            0                0   
4   10001  -1   1   2     8    244            0                0   

   bad_comment_rate  
0               0.0  
1               0.0  
2               0.0  
3               0.0  
4               0.0  


In [9]:
#删除所有时间段都没有购买行为的用户
def filter_buy_users(action_df):
    buy_actions = action_df[action_df["type"] == 4]
    unique_buy_users = buy_actions["user_id"].drop_duplicates()
    return unique_buy_users

def remove_non_buyers(user_df, buy_user_ids):
    print(len(user_df))
    print(len(buy_user_ids))
    filtered_users = user_df[user_df["user_id"].isin(buy_user_ids)]
    print(len(filtered_users))
    return filtered_users

buy_users = filter_buy_users(df_action)
df_user = remove_non_buyers(df_user, buy_users)
print(df_user.head())

105309
29485
29483
    user_id  age  sex  user_lv_cd user_reg_tm
0    200001   -1   -1           5  2016-01-26
4    200005   -1    0           4  2016-01-26
13   200014   -1   -1           4  2013-04-10
14   200015   -1    1           3  2016-01-26
16   200017   -1   -1           4  2016-01-26


In [10]:
def del_user_sku_action_not_in_each_other(df_user, df_sku, df_action):
    df_user_intersection = pd.merge(df_user.loc[:, "user_id"], df_action.loc[:, "user_id"], how="inner", on="user_id")
    df_sku_intersection = pd.merge(df_sku.loc[:, "sku_id"], df_action.loc[:, "sku_id"], how="inner", on="sku_id")
    
    df_action = df_action.loc[(df_action.loc[:, "sku_id"].isin(df_sku_intersection.loc[:, "sku_id"])) \
                              & (df_action.loc[:, "user_id"].isin(df_user_intersection.loc[:, "user_id"]))]
    df_user = df_user.loc[df_user.loc[:, "user_id"].isin(df_action.loc[:, "user_id"])]
    df_sku = df_sku.loc[df_sku.loc[:, "sku_id"].isin(df_action.loc[:, "sku_id"])]
    return df_user, df_sku, df_action
df_user, df_sku, df_action = del_user_sku_action_not_in_each_other(df_user, df_sku, df_action)
print(df_user.head())
print(df_sku.head())
print(df_action.head())

    user_id  age  sex  user_lv_cd user_reg_tm
0    200001   -1   -1           5  2016-01-26
4    200005   -1    0           4  2016-01-26
13   200014   -1   -1           4  2013-04-10
14   200015   -1    1           3  2016-01-26
16   200017   -1   -1           4  2016-01-26
    sku_id  a1  a2  a3  cate  brand  comment_num  has_bad_comment  \
16  100119   2   1   2     8    812            1                0   
27  100174   2   2   1     8    812            2                0   
40  100310   3   2  -1     8    214            0                0   
45  100344   3   1   1     8    214            0                0   
69  100462   3   1   2     8    214            3                0   

    bad_comment_rate  
16               0.0  
27               0.0  
40               0.0  
45               0.0  
69               0.0  
    user_id  sku_id                time  model_id  type  cate  brand
0    266079  138778 2016-01-31 23:59:02       NaN     1     8    403
1    266079  138778 2016-01-31 23

In [11]:
def sync_user_sku_actions(user_df, sku_df, action_df):
    valid_user_ids = pd.merge(user_df[["user_id"]], action_df[["user_id"]], on="user_id", how="inner")["user_id"]
    valid_sku_ids = pd.merge(sku_df[["sku_id"]], action_df[["sku_id"]], on="sku_id", how="inner")["sku_id"]
    
    filtered_actions = action_df[action_df["user_id"].isin(valid_user_ids) & action_df["sku_id"].isin(valid_sku_ids)]
    synced_users = user_df[user_df["user_id"].isin(filtered_actions["user_id"])]
    synced_skus = sku_df[sku_df["sku_id"].isin(filtered_actions["sku_id"])]
    
    return synced_users, synced_skus, filtered_actions

df_user, df_sku, df_action = sync_user_sku_actions(df_user, df_sku, df_action)
print(df_user.head())
print(df_sku.head())
print(df_action.head())

    user_id  age  sex  user_lv_cd user_reg_tm
0    200001   -1   -1           5  2016-01-26
4    200005   -1    0           4  2016-01-26
13   200014   -1   -1           4  2013-04-10
14   200015   -1    1           3  2016-01-26
16   200017   -1   -1           4  2016-01-26
    sku_id  a1  a2  a3  cate  brand  comment_num  has_bad_comment  \
16  100119   2   1   2     8    812            1                0   
27  100174   2   2   1     8    812            2                0   
40  100310   3   2  -1     8    214            0                0   
45  100344   3   1   1     8    214            0                0   
69  100462   3   1   2     8    214            3                0   

    bad_comment_rate  
16               0.0  
27               0.0  
40               0.0  
45               0.0  
69               0.0  
    user_id  sku_id                time  model_id  type  cate  brand
0    266079  138778 2016-01-31 23:59:02       NaN     1     8    403
1    266079  138778 2016-01-31 23

In [12]:
print(len(df_user))
print(len(df_sku))
print(len(df_action))

29430
3403
6589539


In [14]:
def save_data(df_user, df_sku, df_action, filename):
    if not os.path.exists(filename):
        with open(filename, "wb") as f:
            pickle.dump((df_user, df_sku, df_action), f)
            
def load_data(filename):
    with open(filename, "rb") as f:
        df_user, df_sku, df_action = pickle.load(f)
    return df_user, df_sku, df_action

save_data(df_user, df_sku, df_action, "clean_data_without_no_buy.pkl")
df_user, df_sku, df_action = load_data("clean_data_without_no_buy.pkl")

In [17]:
!dir

 驱动器 D 中的卷是 Data
 卷的序列号是 AC3C-F51C

 D:\Python_jupyter\JData高需用户购买预测 的目录

2024/12/15  21:08    <DIR>          .
2024/12/04  22:02    <DIR>          ..
2024/12/15  21:01    <DIR>          .ipynb_checkpoints
2024/11/18  09:11       185,446,670 clean_data_without_no_buy.pkl
2024/11/18  09:09     2,243,573,361 data.pkl
2024/12/15  21:08            21,735 data_clean.ipynb
2024/11/28  17:42           188,063 data_gen_feature_and_train.ipynb
2024/12/15  21:00           322,044 data_visualization.ipynb
2024/12/15  21:07           153,340 data_visualization_plotly.ipynb
2024/11/18  09:55     2,407,392,464 df_dataset.pkl
2024/12/15  21:06    <DIR>          figs
2017/04/03  19:03       521,876,452 JData_Action_201602.csv
2017/04/03  19:05     1,177,389,078 JData_Action_201603.csv
2017/04/03  19:04       600,610,318 JData_Action_201604.csv
2017/04/03  20:23        14,849,315 JData_Comment.csv
2017/04/03  20:34           452,506 JData_Product.csv
2017/04/03  17:23         3,089,469 JData_User.csv
2

In [18]:
df_user.dtypes

user_id       int32
age           int32
sex           int32
user_lv_cd    int32
dtype: object

In [19]:
df_sku.dtypes

sku_id                int64
a1                    int64
a2                    int64
a3                    int64
cate                  int64
brand                 int64
comment_num           int32
has_bad_comment       int32
bad_comment_rate    float32
dtype: object

In [20]:
df_comment.head()

,dt,sku_id,comment_num,has_bad_comment,bad_comment_rate
0,2016-02-01,1000,3,1,0.0417
1,2016-02-01,10000,2,0,0.0000
2,2016-02-01,100011,4,1,0.0376
3,2016-02-01,100018,3,0,0.0000
4,2016-02-01,100020,3,0,0.0000
